In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 386kB/s 
     |████████████████████████████████| 1.5MB 417kB/s 
     |████████████████████████████████| 194kB 4.3MB/s 
     |████████████████████████████████| 235kB 5.9MB/s 
     |████████████████████████████████| 2.2MB 8.9MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


In [ ]:
import glob

ans_path = []
data_path = []
for filename in glob.glob('/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/*.txt'):
  if 'score' in filename:
    ans_path.append(filename)
  else:
    data_path.append(filename)

print(data_path)
print(ans_path)

['/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/29.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/15.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/14.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/28.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/16.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/17.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/13.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/12.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/10.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/11.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/9.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/8.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/5.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/4.txt', '/content/drive/MyDrive/MLG/hw1/hw1_data/Synthetic/5000/6.txt', '/content/drive/MyDrive/MLG/h

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data, ClusterData, ClusterLoader
from torch_geometric.utils import from_networkx

ans_group = []
data_group = []
loader_group = []

for i in range(30):
  ans = {}
  with open(ans_path[i]) as f:
    line = f.readline()
    while line:
      key, value = line.split()
      key = eval(key)
      ans[key] = eval(value)
      line = f.readline()
  ans_group.append(ans)

  G = nx.read_edgelist(data_path[i], nodetype = int)
  edge_index = from_networkx(G).edge_index
  x = torch.tensor([[G.degree(node),1,1] for node in G.nodes], dtype=torch.float)
  y = torch.tensor([ans[node] for node in G.nodes], dtype=torch.float)
  data = Data(edge_index=edge_index, x=x, y=y)
  cluster_data = ClusterData(data, num_parts=128)
  train_loader = ClusterLoader(cluster_data, batch_size=16, shuffle=True)
  data_group.append(data)
  loader_group.append(train_loader)

In [ ]:
import torch
from torch.nn import Linear, GRUCell, ReLU
from torch_geometric.nn import GCNConv
from torch.nn import Sequential as Seq

class Net(torch.nn.Module):
  def __init__(self, L):
    super(Net, self).__init__()
    self.L = L-1 # encoder layer number
    self.data_in = Seq(Linear(data.num_features, 128), ReLU())
    self.Aggregation = GCNConv(128, 128)
    self.Combine = GRUCell(128, 128)
    self.data_out = Seq(Linear(128, 5), ReLU(), Linear(5,1))

  def forward(self, x, edge_index):
    # encoder
    h = self.data_in(x)
    h = h / torch.norm(h) # dimension
    layer = torch.tensor([])
    layer = torch.cat((layer, torch.unsqueeze(h, 0)))
    for _ in range(self.L):
      hn = self.Aggregation(h, edge_index)
      h = self.Combine(h, hn)
      h = h / torch.norm(h)
      layer = torch.cat((layer, torch.unsqueeze(h, 0)))
    z, _ = torch.max(layer, 0)

    # decoder
    y = self.data_out(z)
    y = torch.squeeze(y)
    return y

class customLoss(torch.nn.Module):
  def __init__(self):
    super(customLoss, self).__init__()

  def forward(self, predict, test, edge_index):
    yij = torch.stack([torch.sigmoid(predict[i] - predict[j]) for i, j in zip(edge_index[0], edge_index[1])])
    bij = torch.tensor([torch.sigmoid(test[i] - test[j]) for i, j in zip(edge_index[0], edge_index[1])])
    loss = -bij * torch.log(yij) - (1 - bij) * torch.log(1 - yij)
    loss = torch.sum(loss)
    return loss
    # return torch.nn.BCELoss(yij, bij)

In [ ]:
def top_n_accuracy(train, n, ans):
  num = int(len(train) * n/100)
  train = train.tolist()
  predict = {}
  for i in range(len(train)):
    predict[i] = train[i]

  predict = sorted(predict.items(), key = lambda x: x[1], reverse = True)[:num]
  ans = sorted(ans.items(), key = lambda x: x[1], reverse = True)[:num]
  ans = [item[0] for item in ans]
  hit = 0
  for i in range(num):
    if predict[i][0] in ans:
      hit += 1
  return hit/num

In [ ]:
model = Net(5)
criterion = customLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(i):
  model.train()

  for sub_data in loader_group[i]:
    out = model(sub_data.x, sub_data.edge_index)
    loss = criterion(out, sub_data.y, sub_data.edge_index)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

index = 0
for epoch in range(100):
  train(index)
  predict = model(data_group[index].x, data_group[index].edge_index)
  loss = criterion(predict, data_group[index].y, data_group[index].edge_index)
  print(f'Epoch: {epoch:03d}, Loss: {loss}, Accuracy: {top_n_accuracy(predict, 1, ans_group[index])}')
  print('====predict====')
  print(predict)
  index += 1 if index < 29 else 0

Epoch: 000, Loss: 27701.76953125, Accuracy: 0.26
====predict====
tensor([0.1589, 0.1578, 0.1618,  ..., 0.1442, 0.1441, 0.1441],
       grad_fn=<SqueezeBackward0>)
Epoch: 001, Loss: 27703.19140625, Accuracy: 0.18
====predict====
tensor([0.1660, 0.1803, 0.1748,  ..., 0.1608, 0.1609, 0.1608],
       grad_fn=<SqueezeBackward0>)
Epoch: 002, Loss: 27699.8125, Accuracy: 0.28
====predict====
tensor([0.1976, 0.2068, 0.2037,  ..., 0.1720, 0.1720, 0.1720],
       grad_fn=<SqueezeBackward0>)
Epoch: 003, Loss: 27700.421875, Accuracy: 0.34
====predict====
tensor([0.1961, 0.2085, 0.1940,  ..., 0.1832, 0.1831, 0.1831],
       grad_fn=<SqueezeBackward0>)
Epoch: 004, Loss: 27700.359375, Accuracy: 0.22
====predict====
tensor([0.1969, 0.2056, 0.1991,  ..., 0.1857, 0.1857, 0.1857],
       grad_fn=<SqueezeBackward0>)
Epoch: 005, Loss: 27698.7421875, Accuracy: 0.18
====predict====
tensor([0.1737, 0.2061, 0.1883,  ..., 0.1680, 0.1681, 0.1680],
       grad_fn=<SqueezeBackward0>)
Epoch: 006, Loss: 27703.0859375